In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from matplotlib.ticker import PercentFormatter
from plotnine import *
from sklearn.linear_model import LinearRegression
from scipy import signal
from statsmodels.tsa.seasonal import seasonal_decompose, STL

C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
C:\Users\Jish\AppData\Local\Temp\ipykernel_42140\1519481153.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
os.getcwd()

'C:\\Users\\Jish\\Documents\\DRiV\\Files\\Elasticity\\AMER\\Monroe'

In [3]:
df=pd.read_csv('Strut_Assemblies_Python.csv')
df.head()

,LC,Part/Sku,Year,Month_Num,Month,Sales,Qty,Price,Inflation,AutoPartSales,ConsumerConfidence,Income,Production,RepairPrice,PartsPrice,MilesDriven
0,2a,139104,2019,1,January,120519.96010,869,138.688101,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0
1,2a,139105,2019,1,January,167227.80010,1186,141.001518,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0
2,1a,139112,2019,1,January,22852.63004,136,168.034044,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0
3,2a,171100,2019,1,January,18465.60999,137,134.785474,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0
4,2b,171101L,2019,1,January,12431.65002,93,133.673656,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0


In [4]:
df.dtypes

LC                     object
Part/Sku               object
Year                    int64
Month_Num               int64
Month                  object
Sales                 float64
Qty                     int64
Price                 float64
Inflation             float64
AutoPartSales         float64
ConsumerConfidence    float64
Income                float64
Production            float64
RepairPrice           float64
PartsPrice            float64
MilesDriven           float64
dtype: object

In [5]:
df['Day'] = "1"
df['Year']=df['Year'].astype(str)
df['Month_Num']=df['Month_Num'].astype(str)
# df['Month1']=df['Month_Num']
# df['Year_Month']=pd.to_datetime(df[["Year", "Month", "Day"]])
df['Year_Month']=pd.to_datetime(df.Year+df.Month_Num+df.Day, format="%Y%m%d")
df.rename(columns={'Price':'Price'}, inplace=True)
df.rename(columns={'Units':'Qty'}, inplace=True)
df.rename(columns={'Part/Sku':'Material'}, inplace=True)

In [6]:
df.dtypes

LC                            object
Material                      object
Year                          object
Month_Num                     object
Month                         object
Sales                        float64
Qty                            int64
Price                        float64
Inflation                    float64
AutoPartSales                float64
ConsumerConfidence           float64
Income                       float64
Production                   float64
RepairPrice                  float64
PartsPrice                   float64
MilesDriven                  float64
Day                           object
Year_Month            datetime64[ns]
dtype: object

In [7]:
df.head()

,LC,Material,Year,Month_Num,Month,Sales,Qty,Price,Inflation,AutoPartSales,ConsumerConfidence,Income,Production,RepairPrice,PartsPrice,MilesDriven,Day,Year_Month
0,2a,139104,2019,1,January,120519.96010,869,138.688101,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
1,2a,139105,2019,1,January,167227.80010,1186,141.001518,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
2,1a,139112,2019,1,January,22852.63004,136,168.034044,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
3,2a,171100,2019,1,January,18465.60999,137,134.785474,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
4,2b,171101L,2019,1,January,12431.65002,93,133.673656,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01


In [8]:
df.isna().sum()

LC                       0
Material                 0
Year                     0
Month_Num                0
Month                    0
Sales                    0
Qty                      0
Price                    0
Inflation              962
AutoPartSales         1958
ConsumerConfidence    1958
Income                1958
Production             962
RepairPrice            962
PartsPrice             962
MilesDriven           1958
Day                      0
Year_Month               0
dtype: int64

In [9]:
df=df.dropna(axis=0, subset=('Price', ))

In [10]:
# df['Price'] = df['Price'].astype(float)

In [11]:
df.head()

,LC,Material,Year,Month_Num,Month,Sales,Qty,Price,Inflation,AutoPartSales,ConsumerConfidence,Income,Production,RepairPrice,PartsPrice,MilesDriven,Day,Year_Month
0,2a,139104,2019,1,January,120519.96010,869,138.688101,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
1,2a,139105,2019,1,January,167227.80010,1186,141.001518,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
2,1a,139112,2019,1,January,22852.63004,136,168.034044,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
3,2a,171100,2019,1,January,18465.60999,137,134.785474,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
4,2b,171101L,2019,1,January,12431.65002,93,133.673656,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01


In [12]:
# # Choose the top 80 SKU's
# df=df[df['SKU Tag']=='Top 80%']

In [13]:
df.head()

,LC,Material,Year,Month_Num,Month,Sales,Qty,Price,Inflation,AutoPartSales,ConsumerConfidence,Income,Production,RepairPrice,PartsPrice,MilesDriven,Day,Year_Month
0,2a,139104,2019,1,January,120519.96010,869,138.688101,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
1,2a,139105,2019,1,January,167227.80010,1186,141.001518,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
2,1a,139112,2019,1,January,22852.63004,136,168.034044,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
3,2a,171100,2019,1,January,18465.60999,137,134.785474,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01
4,2b,171101L,2019,1,January,12431.65002,93,133.673656,1.6,7088.0,121.7,1335.65,95.38,290.76,145.59,246517.0,1,2019-01-01


In [14]:
# #Check values in different PH2
# category = df['PH2'].value_counts()
# print("PH2 Category Percentage\n{}\n{}\n PH2 Category values count\n{}\n{}".format(
#       50*"-", (category / len(df.index))[:5] * 100, 50*"-", category[:5]))

In [15]:
#Check values in different Life Cycle
LC = df['LC'].value_counts()
print("LC Percentage\n{}\n{}\n LC values count\n{}\n{}".format(
      50*"-", (LC / len(df.index))[:5] * 100, 50*"-", LC[:5]))

LC Percentage
--------------------------------------------------
LC
2a    35.781293
2b    27.818909
1b    26.680769
3a     6.759864
1a     2.101536
Name: count, dtype: float64
 LC values count
--------------------------------------------------
LC
2a    15562
2b    12099
1b    11604
3a     2940
1a      914
Name: count, dtype: int64


## Without Exclusion of Data

In [16]:
#Aggregate the data
df_sku = df.groupby(['Year_Month','Material']).agg({'Price':'mean','Qty': 'mean' }).reset_index()
# df_sku=df_sku[(df_sku.Year_Month<'2022-10-01') & (df_sku.Year_Month>='2020-07-01')]
df_sku.head()

,Year_Month,Material,Price,Qty
0,2019-01-01,139104,138.688101,869.0
1,2019-01-01,139105,141.001518,1186.0
2,2019-01-01,139112,168.034044,136.0
3,2019-01-01,171100,134.785474,137.0
4,2019-01-01,171101L,133.673656,93.0


In [17]:
## Data Prep using pivot
x_pivot = df_sku.pivot(index= 'Year_Month' ,columns='Material' ,values='Price')
x_values = pd.DataFrame(x_pivot.to_records())
print(x_values)

   Year_Month      139104      139105      139112      153005      153006  \
0  2019-01-01  138.688101  141.001518  168.034044         NaN         NaN   
1  2019-02-01  138.752441  142.354654  157.481439         NaN         NaN   
2  2019-03-01  137.568685  142.320914  164.305519         NaN         NaN   
3  2019-04-01  138.756058  135.921729  166.755099         NaN         NaN   
4  2019-05-01  144.800845  139.572454  165.229258         NaN         NaN   
5  2019-06-01  141.479529  139.980740  161.159348         NaN         NaN   
6  2019-07-01  142.562061  138.890610  168.810277         NaN         NaN   
7  2019-08-01  139.262229  140.352770  170.638148         NaN         NaN   
8  2019-09-01  140.016344  139.138985  180.424418         NaN         NaN   
9  2019-10-01  139.932289  139.638594  175.512297         NaN         NaN   
10 2019-11-01  139.524665  142.774343  168.313702         NaN         NaN   
11 2019-12-01  145.273315  147.781289  182.764571         NaN         NaN   

In [18]:
percent_missing_x = x_values.isnull().sum() * 100 / len(x_values)
missing_value_df_x = pd.DataFrame({'column_name': x_values.columns,
                                 'percent_missing': percent_missing_x})
missing_value_df_x.sort_values(by=['percent_missing'])

,column_name,percent_missing
Year_Month,Year_Month,0.000000
172780L,172780L,0.000000
172780R,172780R,0.000000
172781,172781,0.000000
172782,172782,0.000000
...,...,...
172755,172755,77.777778
281525,281525,80.000000
253006,253006,80.000000
272413,272413,84.444444


In [19]:
mod_missing_value_df_x=missing_value_df_x[missing_value_df_x.percent_missing<=20]
mod_missing_value_df_x

,column_name,percent_missing
Year_Month,Year_Month,0.000000
139104,139104,0.000000
139105,139105,0.000000
139112,139112,0.000000
153026,153026,4.444444
...,...,...
572888,572888,4.444444
572889,572889,4.444444
672526,672526,0.000000
672527,672527,0.000000


In [20]:
mod_missing_value_df_x=mod_missing_value_df_x.reset_index(drop=True)
mod_missing_value_df_x['column_name']

0      Year_Month
1          139104
2          139105
3          139112
4          153026
          ...    
865        572888
866        572889
867        672526
868        672527
869        672665
Name: column_name, Length: 870, dtype: object

In [21]:
x_values_mod = x_values[x_values.columns[x_values.columns.isin(mod_missing_value_df_x['column_name'].astype('str'))]]
x_values_mod

,Year_Month,139104,139105,139112,153026,153027,153028,153029,153030,153031,...,571377L,571377R,572526,572527,572665,572888,572889,672526,672527,672665
0,2019-01-01,138.688101,141.001518,168.034044,NaN,NaN,NaN,NaN,NaN,NaN,...,176.028077,172.759583,173.362174,169.773809,164.828695,NaN,NaN,162.440000,163.611852,95.129464
1,2019-02-01,138.752441,142.354654,157.481439,NaN,NaN,NaN,NaN,NaN,NaN,...,170.154791,171.031738,169.454212,164.307334,157.230324,NaN,NaN,178.755910,179.375001,96.027392
2,2019-03-01,137.568685,142.320914,164.305519,205.760002,162.520000,NaN,NaN,212.669999,169.420000,...,172.190921,170.364147,160.260303,155.961875,161.279514,109.750000,109.750000,160.082223,163.680001,95.770592
3,2019-04-01,138.756058,135.921729,166.755099,191.883333,160.254375,158.884286,173.140000,189.434442,159.907000,...,169.245157,167.790309,172.593530,173.122354,148.326099,147.033330,147.707497,172.077318,167.825527,100.736947
4,2019-05-01,144.800845,139.572454,165.229258,209.829999,154.839678,155.050000,208.908334,201.884999,148.910000,...,174.207762,173.333530,157.856155,156.988001,155.572937,141.444285,140.611818,164.691225,171.296038,98.012380
5,2019-06-01,141.479529,139.980740,161.159348,213.088571,156.808049,201.428749,152.180000,183.666296,177.358886,...,180.543734,182.673334,154.135405,154.858125,156.145889,156.169997,149.149998,165.803877,165.767368,101.656861
6,2019-07-01,142.562061,138.890610,168.810277,198.343889,168.983793,175.776000,177.645000,186.515265,182.105000,...,171.190294,171.519079,176.863666,173.530356,155.063333,137.633333,137.633333,172.520000,174.030000,101.500367
7,2019-08-01,139.262229,140.352770,170.638148,209.987691,184.679571,198.131250,173.685556,172.369047,175.593846,...,171.973290,170.174935,173.480001,172.747693,165.369524,139.058333,139.108462,160.565601,162.813112,102.367852
8,2019-09-01,140.016344,139.138985,180.424418,203.122050,164.070477,192.287201,155.600000,222.637334,170.695000,...,170.880588,172.776923,165.998685,167.817251,152.436847,149.260555,149.260555,162.013913,161.062826,107.080520
9,2019-10-01,139.932289,139.638594,175.512297,212.603404,165.254418,192.770541,163.547500,220.228331,164.074285,...,169.107619,163.847846,163.090810,168.483250,162.121840,150.862143,149.830000,165.738334,165.113784,102.060428


In [22]:
y_pivot = df_sku.pivot( index = 'Year_Month',columns='Material', values='Qty')
y_values = pd.DataFrame(y_pivot.to_records())
print(y_values)

   Year_Month  139104  139105  139112  153005  153006  153026  153027  153028  \
0  2019-01-01   869.0  1186.0   136.0     NaN     NaN     NaN     NaN     NaN   
1  2019-02-01   967.0  1285.0   153.0     NaN     NaN     NaN     NaN     NaN   
2  2019-03-01  1582.0  2275.0   241.0     NaN     NaN     6.0     6.0     NaN   
3  2019-04-01  1276.0  2042.0   202.0     NaN     NaN     9.0    32.0     7.0   
4  2019-05-01  1149.0  1854.0   229.0     NaN     NaN    11.0    31.0     2.0   
5  2019-06-01  1463.0  2405.0   276.0     NaN     NaN    21.0    41.0    16.0   
6  2019-07-01  1223.0  2015.0   217.0     NaN     NaN    36.0    29.0    15.0   
7  2019-08-01  1499.0  2344.0   297.0     NaN     NaN    52.0    70.0    32.0   
8  2019-09-01  1228.0  1801.0   206.0     NaN     NaN    39.0    42.0    25.0   
9  2019-10-01  1241.0  1734.0   209.0     NaN     NaN    47.0    43.0    37.0   
10 2019-11-01  1385.0  1727.0   235.0     NaN     NaN    34.0    35.0    29.0   
11 2019-12-01   887.0  1164.

In [23]:
percent_missing_y = y_values.isnull().sum() * 100 / len(x_values)
missing_value_df_y = pd.DataFrame({'column_name': y_values.columns,
                                 'percent_missing': percent_missing_y})
missing_value_df_y.sort_values(by=['percent_missing'])

,column_name,percent_missing
Year_Month,Year_Month,0.000000
172780L,172780L,0.000000
172780R,172780R,0.000000
172781,172781,0.000000
172782,172782,0.000000
...,...,...
172755,172755,77.777778
281525,281525,80.000000
253006,253006,80.000000
272413,272413,84.444444


In [24]:
mod_missing_value_df_y=missing_value_df_y[missing_value_df_y.percent_missing<=20]

In [25]:
mod_missing_value_df_y=mod_missing_value_df_y.reset_index(drop=True)
mod_missing_value_df_y['column_name']

0      Year_Month
1          139104
2          139105
3          139112
4          153026
          ...    
865        572888
866        572889
867        672526
868        672527
869        672665
Name: column_name, Length: 870, dtype: object

In [26]:
y_values_mod = y_values[y_values.columns[y_values.columns.isin(mod_missing_value_df_y['column_name'].astype('str'))]]
y_values_mod

,Year_Month,139104,139105,139112,153026,153027,153028,153029,153030,153031,...,571377L,571377R,572526,572527,572665,572888,572889,672526,672527,672665
0,2019-01-01,869.0,1186.0,136.0,NaN,NaN,NaN,NaN,NaN,NaN,...,52.0,48.0,23.0,21.0,92.0,NaN,NaN,25.0,27.0,112.0
1,2019-02-01,967.0,1285.0,153.0,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,46.0,19.0,15.0,123.0,NaN,NaN,22.0,30.0,115.0
2,2019-03-01,1582.0,2275.0,241.0,6.0,6.0,NaN,NaN,4.0,4.0,...,76.0,82.0,33.0,32.0,226.0,1.0,1.0,72.0,79.0,152.0
3,2019-04-01,1276.0,2042.0,202.0,9.0,32.0,7.0,1.0,9.0,10.0,...,64.0,65.0,17.0,17.0,123.0,6.0,4.0,41.0,38.0,154.0
4,2019-05-01,1149.0,1854.0,229.0,11.0,31.0,2.0,12.0,4.0,6.0,...,67.0,68.0,39.0,40.0,109.0,14.0,11.0,49.0,53.0,147.0
5,2019-06-01,1463.0,2405.0,276.0,21.0,41.0,16.0,2.0,27.0,18.0,...,75.0,78.0,37.0,32.0,107.0,5.0,6.0,49.0,57.0,153.0
6,2019-07-01,1223.0,2015.0,217.0,36.0,29.0,15.0,4.0,19.0,16.0,...,68.0,76.0,30.0,28.0,105.0,9.0,9.0,35.0,39.0,109.0
7,2019-08-01,1499.0,2344.0,297.0,52.0,70.0,32.0,18.0,42.0,26.0,...,76.0,77.0,37.0,39.0,126.0,18.0,13.0,50.0,45.0,121.0
8,2019-09-01,1228.0,1801.0,206.0,39.0,42.0,25.0,2.0,30.0,16.0,...,68.0,65.0,38.0,40.0,111.0,18.0,18.0,46.0,46.0,77.0
9,2019-10-01,1241.0,1734.0,209.0,47.0,43.0,37.0,8.0,24.0,21.0,...,63.0,65.0,37.0,40.0,125.0,14.0,13.0,30.0,37.0,117.0


In [27]:
results_values_raw = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
# results_values_detrended = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
results_values_deseasonalized = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
results_values_log_raw = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
# results_values_log_detrended = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}
results_values_log_deseasonalized = { "name": [], "price_elasticity": [], "price_mean": [],"quantity_mean": [], "intercept": [],  "t_score":[], "slope": [], "coefficient_pvalue" : [], "rsquared" : []}

In [28]:
# for column in x_values_mod.columns[1:25]:
#     column_points = []
#     for i in range(len(x_values_mod[column])):
#         if not np.isnan(x_values_mod[column][i]) and not np.isnan(y_values_mod[column][i]):
#             # column_points.append((x_values_mod[column][i], y_values_mod[column][i]))
#             column_points.append((column,x_values_mod['Year_Month'][i],x_values_mod[column][i], y_values_mod[column][i]))
#     # df = pd.DataFrame(list(column_points), columns= ['x_value', 'y_value'])
#     df = pd.DataFrame(list(column_points), columns= ['Material','Date','x_value', 'y_value'])
    
#     ## Deseasonalize
#     # df.reset_index(inplace=False)
#     df = df.set_index('Date')
#     df.sort_index(inplace=True)
#     df = df.resample('MS').asfreq().fillna(0)
#     # print(df.index.freq)
#     print(df)
#     res = seasonal_decompose(df.y_value, extrapolate_trend='freq')
#     # df['y_value'] = res.trend
#     df['y_value_deseasonalized'] = df.y_value.values - res.seasonal
#     df = pd.DataFrame(df)
#     # print(df)
#     # sns.lineplot(data = df[['y_value','y_value_deseasonalized']])
#     # plt.title('Deseasonalized Plot')
#     # plt.show()
    
#     # math.log(x)
#     df['log_x_value'] = np.where(df['x_value'] > 0, np.log(df['x_value']) , 0)
#     df['log_y_value'] = np.where(df['y_value'] > 0, np.log(df['y_value']) , 0) 
#     df['log_y_value_deseasonalized'] = np.where(df['y_value_deseasonalized'] > 0, np.log(df['y_value_deseasonalized']) , 0)
#     # df['x_value'] = df['x_value'].apply(lambda x:x if x>=0 else 0) 
#     # df['log_y_value'] = np.log(df['y_value'])
#     # df['log_y_value_deseasonalized'] = np.log(df['y_value_deseasonalized'])
#     print(df)

In [29]:
for column in x_values_mod.columns[1:]:
    column_points = []
    for i in range(len(x_values_mod[column])):
        if not np.isnan(x_values_mod[column][i]) and not np.isnan(y_values_mod[column][i]):
            # column_points.append((x_values_mod[column][i], y_values_mod[column][i]))
            column_points.append((column,x_values_mod['Year_Month'][i],x_values_mod[column][i], y_values_mod[column][i]))
    # df = pd.DataFrame(list(column_points), columns= ['x_value', 'y_value'])
    df = pd.DataFrame(list(column_points), columns= ['Material','Date','x_value', 'y_value'])
    
    ## Deseasonalize
    # df.reset_index(inplace=False)
    df = df.set_index('Date')
    df.sort_index(inplace=True)
    df = df.resample('MS').asfreq().fillna(0)
    # print(df.index.freq)
    # print(df)
    res = seasonal_decompose(df.y_value, extrapolate_trend='freq')
    # df['y_value'] = res.trend
    df['y_value_deseasonalized'] = df.y_value.values - res.seasonal
    df = pd.DataFrame(df)
    # print(df)
    # sns.lineplot(data = df[['y_value','y_value_deseasonalized']])
    # plt.title('Deseasonalized Plot')
    # plt.show()
    
    # math.log(x)
    df['log_x_value'] = np.where(df['x_value'] > 0, np.log(df['x_value']) , 0)
    df['log_y_value'] = np.where(df['y_value'] > 0, np.log(df['y_value']) , 0) 
    df['log_y_value_deseasonalized'] = np.where(df['y_value_deseasonalized'] > 0, np.log(df['y_value_deseasonalized']) , 0)
    # df['x_value'] = df['x_value'].apply(lambda x:x if x>=0 else 0) 
    # df['log_y_value'] = np.log(df['y_value'])
    # df['log_y_value_deseasonalized'] = np.log(df['y_value_deseasonalized'])
    # print(df)


    #Raw Model
    x_value = df['x_value']
    y_value = df['y_value']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_raw["name"].append(column)
    results_values_raw["price_elasticity"].append(price_elasticity)
    results_values_raw["price_mean"].append(mean_price)
    results_values_raw["quantity_mean"].append(mean_quantity)
    results_values_raw["intercept"].append(intercept)
    results_values_raw['t_score'].append(t_score)
    results_values_raw["slope"].append(slope)
    results_values_raw["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_raw["rsquared"].append(rsquared*100)
    
    
    #Deseasonalized Model
    x_value = df['x_value']
    y_value = df['y_value_deseasonalized']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_deseasonalized["name"].append(column)
    results_values_deseasonalized["price_elasticity"].append(price_elasticity)
    results_values_deseasonalized["price_mean"].append(mean_price)
    results_values_deseasonalized["quantity_mean"].append(mean_quantity)
    results_values_deseasonalized["intercept"].append(intercept)
    results_values_deseasonalized['t_score'].append(t_score)
    results_values_deseasonalized["slope"].append(slope)
    results_values_deseasonalized["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_deseasonalized["rsquared"].append(rsquared*100)

   #Log Raw Model
    x_value = df['log_x_value']
    y_value = df['log_y_value']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_log_raw["name"].append(column)
    results_values_log_raw["price_elasticity"].append(price_elasticity)
    results_values_log_raw["price_mean"].append(mean_price)
    results_values_log_raw["quantity_mean"].append(mean_quantity)
    results_values_log_raw["intercept"].append(intercept)
    results_values_log_raw['t_score'].append(t_score)
    results_values_log_raw["slope"].append(slope)
    results_values_log_raw["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_log_raw["rsquared"].append(rsquared*100)
    
    
    #log Deseasonalized Model
    x_value = df['log_x_value']
    y_value = df['log_y_value_deseasonalized']
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    #(Null Hypothesis test) Coefficient with a p value less than 0.05
#     if result.f_pvalue < 0.05:
        
    rsquared = result.rsquared
    coefficient_pvalue = result.f_pvalue
    intercept, slope = result.params
    mean_price = np.mean(x_value)
    mean_quantity = np.mean(y_value)
    tintercept, t_score = result.tvalues

    #Price elasticity Formula
    price_elasticity = (slope)*(mean_price/mean_quantity)    

    #Append results into dictionary for dataframe
    results_values_log_deseasonalized["name"].append(column)
    results_values_log_deseasonalized["price_elasticity"].append(price_elasticity)
    results_values_log_deseasonalized["price_mean"].append(mean_price)
    results_values_log_deseasonalized["quantity_mean"].append(mean_quantity)
    results_values_log_deseasonalized["intercept"].append(intercept)
    results_values_log_deseasonalized['t_score'].append(t_score)
    results_values_log_deseasonalized["slope"].append(slope)
    results_values_log_deseasonalized["coefficient_pvalue"].append(coefficient_pvalue)
    results_values_log_deseasonalized["rsquared"].append(rsquared*100)
    
df_raw = pd.DataFrame.from_dict(results_values_raw)
df_raw['ID'] = '1-Raw'
df_raw_deseasonalized = pd.DataFrame.from_dict(results_values_deseasonalized)
df_raw_deseasonalized['ID'] = '4-results_values_deseasonalized'
df_raw_log = pd.DataFrame.from_dict(results_values_log_raw)
df_raw_log['ID'] = '3-Log_Raw'
df_raw_deseasonalized_log = pd.DataFrame.from_dict(results_values_log_deseasonalized)
df_raw_deseasonalized_log['ID'] = '2-Log_results_values_deseasonalized'
final_df = pd.concat([df_raw,df_raw_deseasonalized,df_raw_log,df_raw_deseasonalized_log])
# final_df = pd.concat([df_raw])
df_elasticity =final_df[['ID','name','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared']]

C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\

C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\li

C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\li

C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
C:\Users\Jish\anaconda3\lib\

In [30]:
# for column in x_values_mod.columns[1:]:
#     column_points = []
#     for i in range(len(x_values_mod[column])):
#         if not np.isnan(x_values_mod[column][i]) and not np.isnan(y_values_mod[column][i]):
#             # column_points.append((x_values_mod[column][i], y_values_mod[column][i]))
#             column_points.append((i+1,x_values_mod['Year_Month'][i],x_values_mod[column][i], y_values_mod[column][i]))
#     # df = pd.DataFrame(list(column_points), columns= ['x_value', 'y_value'])
#     df = pd.DataFrame(list(column_points), columns= ['Date_Index','Date','x_value', 'y_value'])
    
#     ## Deseasonalize
#     # df.reset_index(inplace=False)
#     df = df.set_index('Date')
#     df.sort_index(inplace=True)
#     df = df.resample('MS').asfreq().fillna(0)
#     # df.asfreq('MS')
#     print(df.index.freq)
#     print(df)
#     res = seasonal_decompose(df.y_value, extrapolate_trend='freq')
#     # df['y_value'] = res.trend
#     df['y_value_deseasonalized'] = df.y_value.values - res.seasonal
#     df = pd.DataFrame(df)
#     print(df)

In [31]:
final_df.sort_values(by=['rsquared'], ascending=False)

,name,price_elasticity,price_mean,quantity_mean,intercept,t_score,slope,coefficient_pvalue,rsquared,ID
791,273031R,5.053882,180.165516,52.047233,-210.993327,12.142444,1.459994,1.739012e-15,77.420586,4-results_values_deseasonalized
790,273031L,5.193010,180.970740,51.823829,-217.297857,11.826628,1.487101,4.183339e-15,76.485917,4-results_values_deseasonalized
790,273031L,7.535742,5.192644,3.745083,-24.476898,11.721520,5.434993,5.618638e-15,76.163292,3-Log_Raw
791,273031R,7.420807,5.187776,3.742442,-24.029500,11.602405,5.353342,7.862468e-15,75.790442,3-Log_Raw
16,171109,-5.720989,4.983564,3.091201,20.775927,-11.586024,-3.548610,8.235490e-15,75.738556,2-Log_results_values_deseasonalized
...,...,...,...,...,...,...,...,...,...,...
504,172896,0.006546,140.273562,222.456142,220.999921,0.007732,0.010381,9.938668e-01,0.000139,4-results_values_deseasonalized
610,271146,0.002436,115.023190,6.972738,6.955752,0.007014,0.000148,9.944369e-01,0.000117,4-results_values_deseasonalized
232,172214,0.003144,5.236249,3.989498,3.976953,0.004347,0.002396,9.965518e-01,0.000044,3-Log_Raw
692,272401,-0.003940,4.754730,4.154091,4.170459,-0.004248,-0.003443,9.966303e-01,0.000042,3-Log_Raw


In [32]:
# good=np.where((final_df['rsquared']>=30) & (final_df['ID'].str.contains('1-Raw')))
# good=final_df.query('rsquared>=30 & ID.str.contains("1-Raw")')
good=final_df[final_df['rsquared']>=30]
good.sort_values(by=['rsquared',],ascending=False)

,name,price_elasticity,price_mean,quantity_mean,intercept,t_score,slope,coefficient_pvalue,rsquared,ID
791,273031R,5.053882,180.165516,52.047233,-210.993327,12.142444,1.459994,1.739012e-15,77.420586,4-results_values_deseasonalized
790,273031L,5.193010,180.970740,51.823829,-217.297857,11.826628,1.487101,4.183339e-15,76.485917,4-results_values_deseasonalized
790,273031L,7.535742,5.192644,3.745083,-24.476898,11.721520,5.434993,5.618638e-15,76.163292,3-Log_Raw
791,273031R,7.420807,5.187776,3.742442,-24.029500,11.602405,5.353342,7.862468e-15,75.790442,3-Log_Raw
16,171109,-5.720989,4.983564,3.091201,20.775927,-11.586024,-3.548610,8.235490e-15,75.738556,2-Log_results_values_deseasonalized
...,...,...,...,...,...,...,...,...,...,...
666,272261,-1.894944,4.916428,4.768884,13.805654,-4.322188,-1.838076,8.981123e-05,30.286820,2-Log_results_values_deseasonalized
312,172359,-4.953897,5.313407,2.531929,15.074848,-4.319631,-2.360617,9.053849e-05,30.261842,3-Log_Raw
246,172240,-4.615575,4.954974,4.119611,23.133983,-4.318069,-3.837432,9.098584e-05,30.246573,2-Log_results_values_deseasonalized
785,272971,3.248604,5.222126,3.390983,-7.624979,4.307852,2.109478,9.396441e-05,30.146714,2-Log_results_values_deseasonalized


In [33]:
len(good)

654

In [34]:
print(final_df.query('rsquared>=30 & name.str.contains("273031L")'))

        name  price_elasticity  price_mean  quantity_mean   intercept  \
790  273031L          5.119335  180.970740      52.222222 -215.120836   
790  273031L          5.193010  180.970740      51.823829 -217.297857   
790  273031L          7.535742    5.192644       3.745083  -24.476898   
790  273031L          8.108039    5.192644       3.715148  -26.407420   

       t_score     slope  coefficient_pvalue   rsquared  \
790  10.786586  1.477272        8.249950e-14  73.015447   
790  11.826628  1.487101        4.183339e-15  76.485917   
790  11.721520  5.434993        5.618638e-15  76.163292   
790  10.121562  5.801008        5.974512e-13  70.435766   

                                      ID  
790                                1-Raw  
790      4-results_values_deseasonalized  
790                            3-Log_Raw  
790  2-Log_results_values_deseasonalized  


In [35]:
final_df.to_csv("jay_new_sku.csv")